# Vaihe 1

### Kiinnostavat tutkimuskysymykset
1. Mitkä genret ovat olleet suosittuja milloin?
2. Millaisia ovat suositut näyttelijäparit?
3. Korrelaatio arvioiden ja tuoton välillä. Voiko elokuva olla huonosti arvioitu mutta silti tuottoisa tai toisin päin?
4. Voiko elokuvan tuottoa ennustaa annetuilla parametreilla?

# Vaihe 2

### Aineiston kuvaus
Aineisto löydettiin Kagglesta ja se sisältää tietoa elokuvista IMDB palvelussa.

Linkki aineistoon: [LINKKI](https://www.kaggle.com/datasets/ashpalsingh1525/imdb-movies-dataset)


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# set display options
pd.set_option('display.max_colwidth', None)

dataset = pd.read_csv('Datasets/imdb_movies.csv')

# Print dataframe columns
print("Aineiston otsikot:")
print(dataset.columns)
print("Otsikoiden tyypit:")
print(dataset.dtypes)
print("Aineiston muoto:")
print(dataset.shape)
amount_of_rows_pre = dataset.shape[0]

Aineiston otsikot:
Index(['names', 'date_x', 'score', 'genre', 'overview', 'crew', 'orig_title',
       'status', 'orig_lang', 'budget_x', 'revenue', 'country'],
      dtype='object')
Otsikoiden tyypit:
names          object
date_x         object
score         float64
genre          object
overview       object
crew           object
orig_title     object
status         object
orig_lang      object
budget_x      float64
revenue       float64
country        object
dtype: object
Aineiston muoto:
(10178, 12)


#### Aineiston otsikot:
- names:        Elokuvan nimi englanniksi
- date_x:       Elokuvan julkaisupäivämäärä
- score:        IMDB-käyttäjien antaman pistemäärän keskiarvo 
- genre:        Elokuvan genre
- overview:     Elokuvan kuvaus englanniksi
- crew:         Elokuvan näyttelijät ja heidän esittämät hahmot. esim. (näyttelijä, hahmo, näyttelijä, hahmo)
- orig_title:   Elokuvan nimi alkuperäisellä kielellä
- status:       Elokuvan tilanne. (Kaikki havainnot ovat julkaistu)
- orig_lang:    Elokuvan alkuperäiskieli
- budget_x:     Elokuvan budjetti (USD)
- revenue:      Elokuvan bruttotuotto (USD)
- country:      Elokuvan julkaisumaa

Aineistossa on siis 12 otsikkoa ja 10178 havaintoa.


Osa aineiston datasta on epäilyttävää. Esimerkiksi 144 elokuvista on saanut 0.0 arvostelun ja niiden kaikkien tuotto on tasan 175,269,998.80 dollaria.
Aineistossa on myös 9 elokuvaa 100.0 arvioinnilla, niistä jokaisen budjetti on 201,000,000.0 dollaria ja tuotto tasan 156,932,3843.8 dollaria.

Aineiston julkaisija on ilmeisesti käyttänyt jotain knn keinoa aineiston täyttämiseen. Tämä täytyy ottaa huomioon myöhemmin.



# Vaihe 3
## Aineiston esikäsittely

#### Huonojen havaintojen poisto

In [37]:
# Tässä poistetaan ne havainnot joissa on tyhjiä arvoja, arvostelu on 0.0 tai 100.0
dataset = dataset.dropna()
dataset = dataset[~dataset['score'].isin([1.0, 100.0])]
print(dataset.shape)
amount_of_rows_post = dataset.shape[0]
print(f"Poistettu {amount_of_rows_pre - amount_of_rows_post} riviä, joissa oli tyhjiä arvoja tai arvostelu oli 0.0 tai 100.0")

(10044, 12)
Poistettu 134 riviä, joissa oli tyhjiä arvoja tai arvostelu oli 0.0 tai 100.0


#### Avataan näyttelijät
Emme ole kiinnostuneita näyttelijöiden hahmoista, joten poistetaan ne aineistosta.

In [38]:
print("1. elokuvan näyttelijät:")
print(dataset['crew'].head(1))

1. elokuvan näyttelijät:
0    Michael B. Jordan, Adonis Creed, Tessa Thompson, Bianca Taylor, Jonathan Majors, Damien Anderson, Wood Harris, Tony 'Little Duke' Evers, Phylicia Rashād, Mary Anne Creed, Mila Davis-Kent, Amara Creed, Florian Munteanu, Viktor Drago, José Benavidez Jr., Felix Chavez, Selenis Leyva, Laura Chavez
Name: crew, dtype: object


In [39]:
# Tässä avataan näyttelijät. Näyttelijän hahmo on listattu näyttelijän jälkeen. (esim. Tom Cruise, Ethan Hunt, ...)
dataset['crew'] = dataset['crew'].str.split(', ')
# Poista joka toinen alkio
dataset['crew'] = dataset['crew'].apply(lambda x: x[::2])
print("1. elokuvan näyttelijät:")
print(dataset['crew'].head(1))

1. elokuvan näyttelijät:
0    [Michael B. Jordan, Tessa Thompson, Jonathan Majors, Wood Harris, Phylicia Rashād, Mila Davis-Kent, Florian Munteanu, José Benavidez Jr., Selenis Leyva]
Name: crew, dtype: object


Crew -sarake sisältää siis nyt vain näyttelijät ilman hahmoja

# Vaihe 4
### Aineiston analysointi

In [40]:
#  JOKAISESSA ANALYSOINTIKOHDASSA TEHDÄÄN AINEISTOSTA KOPIO, ETTEI MYÖHEMMÄT KOHDAT MUUTU

#### Mitkä genret ovat suosittuja milloin

Found 19 unique genres
   year  Action  Adventure  Animation  Comedy  Crime  Documentary  Drama  \
0  2023       1          0          0       0      0            0      1   
1  2022       1          1          0       0      0            0      0   
2  2023       0          1          1       1      0            0      0   
3  2023       0          1          1       1      0            0      0   
4  2023       1          0          0       0      0            0      0   

   Family  Fantasy  History  Horror  Music  Mystery  Romance  Science Fiction  \
0       0        0        0       0      0        0        0                0   
1       0        0        0       0      0        0        0                1   
2       1        1        0       0      0        0        0                0   
3       1        1        0       0      0        0        0                0   
4       0        0        0       0      0        0        0                0   

   TV Movie  Thriller  War  Weste

In [41]:
dataset_genre = dataset.copy()

# Drop description column
dataset_genre = dataset_genre.drop(columns=['overview'])

# Extract year from the release date
dataset_genre['date_x'] = pd.to_datetime(dataset_genre['date_x'])
dataset_genre['year'] = dataset_genre['date_x'].dt.year

# Extract all individual genres from the genre column
all_genres = []
for genre_string in dataset_genre['genre']:
    # Split on comma and remove any non-standard whitespace
    genres = [g.strip() for g in genre_string.split(',')]
    all_genres.extend(genres)

# Get unique genres and sort them
unique_genres = sorted(list(set(all_genres)))
print(f"Found {len(unique_genres)} unique genres")

# Create a new dataframe with a column for each genre
genres = pd.DataFrame(index=dataset_genre.index)
for genre in unique_genres:
    genres[genre] = dataset_genre['genre'].apply(lambda x: 1 if genre in x else 0)

# Join the original dataset with the genre dataframe
dataset_genre = dataset_genre.join(genres)

# Drop unnecessary columns
dataset_genre = dataset_genre.drop(columns=['genre', 'crew', 'names', 'orig_title', 'status', 'orig_lang', 'budget_x', 'score', 'date_x', 'revenue', 'country'])

# Print the first few rows of the dataset
print(dataset_genre.head())
print(dataset_genre.shape)

# Change the year of a movie to the decade it was released in
for row in dataset_genre.itertuples():
    dataset_genre.at[row.Index, 'year'] = row.year - (row.year % 10)


# --- PLOT ---
# Group by decade and sum the genre columns
dataset_genre = dataset_genre.groupby('year').sum()

# Create a colormap for the genres
colormap = plt.cm.tab20

# Plot the data
plt.figure(figsize=(20, 10))
bar_width = 0.75 / len(dataset_genre.columns)
x = np.arange(len(dataset_genre.index))

for i, genre in enumerate(dataset_genre.columns):
    plt.bar(x + i * bar_width, dataset_genre[genre],
            width=bar_width, label=genre, color=colormap(i % 20))

plt.xticks(x + bar_width * len(dataset_genre.columns) / 2 - bar_width/2, dataset_genre.index)
plt.legend(loc='upper left', ncol=2)
plt.title('Movies by genre and decade')
plt.xlabel('Decade')
plt.ylabel('Number of movies')
plt.yscale('log')
plt.tight_layout()
plt.show()

##### Analyysi:
Kaavio on logaritminen. Voidaan päätellä, että ylipäänsä elokuvien määrä on lisääntynyt vuosien saatossa. 2010 -vuonna elokuvia tehtiin noin 1000 -kertainen määrä vuoteen 1900 verrattuna. Myös genrejen kirjo on kasvanut.

#### Millaisia ovat suositut näyttelijäparit?

In [42]:
from itertools import combinations
from collections import Counter
from collections import defaultdict
import pandas as pd
import numpy as np

# Copy the dataset 
dataset_actors = dataset.copy()

# Extract all individual genres from the genre column
all_genres = []
for genre_string in dataset_actors['genre']:
    # Split on comma and remove any non-standard whitespace
    genres = [g.strip() for g in genre_string.split(',')]
    all_genres.extend(genres)

# Get unique genres and sort them
unique_genres = sorted(list(set(all_genres)))
print(f"Found {len(unique_genres)} unique genres")

# Create a new dataframe with a column for each genre
genres = pd.DataFrame(index=dataset_actors.index)
for genre in unique_genres:
    genres[genre] = dataset_actors['genre'].apply(lambda x: 1 if genre in x else 0)

# Join the original dataset with the genre dataframe
dataset_actors = dataset_actors.join(genres)

# Drop unnecessary columns
dataset_actors = dataset_actors.drop(columns=['names', 'orig_title', 'status', 'orig_lang', 'date_x', 'overview', 'genre'])
# Analyzing actor combinations and their impact on movie metrics

import matplotlib.pyplot as plt

# Create a dictionary to store the average score for each actor combination
actor_scores = defaultdict(list)

# Iterate through each row in the dataset
for _, row in dataset_actors.iterrows():
    crew = row['crew']
    score = row['score']
    revenue = row['revenue']
    budget = row['budget_x']
    country = row.get('country', '')  # Get country info if available
    
    # Skip Korean films
    if country in ['KR', 'South Korea', 'Korea']:
        continue
    
    # Get all possible pairs of actors in the crew
    actor_combinations = list(combinations(crew, 2))
    
    # Store the score for each actor combination
    for combo in actor_combinations:
        actor_scores[combo].append({'score': score, 'revenue': revenue, 'budget': budget})

# Calculate the average score for each actor combination
actor_combination_scores = {}
for combo, movies in actor_scores.items():
    if len(movies) >= 2:  # Only consider combinations that appear in at least 2 movies
        avg_score = sum(movie['score'] for movie in movies) / len(movies)
        actor_combination_scores[combo] = avg_score

# Convert to pandas Series for easier analysis
actor_combination_scores = pd.Series(actor_combination_scores)

# Find top actor combinations by score
top_combinations = actor_combination_scores.nlargest(20)
print("\nTop 20 actor combinations by average score:")
print(top_combinations)

# Find best actor pair
best_combo = top_combinations.index[0]
best_score = top_combinations.iloc[0]
print(f"\nBest actor combination: {best_combo[0]} and {best_combo[1]} with average score: {best_score:.1f}")

# Calculate how many times each actor pair appears together
actor_pair_counts = Counter(tuple(sorted(pair)) for _, row in dataset_actors.iterrows() 
                           for pair in combinations(row['crew'], 2))

# Find actor pairs that appear in multiple movies together
frequent_pairs = {pair: count for pair, count in actor_pair_counts.items() if count > 1}
print(f"\nFound {len(frequent_pairs)} actor pairs that appear in multiple movies together")

# Get the top 10 most frequent actor pairs
top_10_pairs = sorted(frequent_pairs.items(), key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 actor pairs by number of movies together:")
for pair, count in top_10_pairs:
    # Check if the pair exists in actor_scores and has movies before calculating averages
    if pair in actor_scores and len(actor_scores[pair]) > 0:
        avg_metrics = {
            'score': sum(movie['score'] for movie in actor_scores[pair]) / len(actor_scores[pair]),
            'revenue': sum(movie['revenue'] for movie in actor_scores[pair]) / len(actor_scores[pair]),
            'budget': sum(movie['budget'] for movie in actor_scores[pair]) / len(actor_scores[pair])
        }
    print(f"{pair[0]} and {pair[1]}: {count} movies together")
    print(f"  Average score: {avg_metrics['score']:.1f}")
    print(f"  Average revenue: ${avg_metrics['revenue']/1000000:.1f}M")
    print(f"  Average budget: ${avg_metrics['budget']/1000000:.1f}M")

# Create a dataframe for detailed analysis of actor pairs with minimum occurrences
min_occurrences = 5
actor_pair_combined = pd.DataFrame([
    {
        'actor1': combo[0],
        'actor2': combo[1],
        'count': len(movies),
        'success_count': sum(1 for movie in movies if movie['score'] > dataset_actors['score'].mean()),
        'average_score': sum(movie['score'] for movie in movies) / len(movies)
    }
    for combo, movies in actor_scores.items()
    if len(movies) >= min_occurrences
])

print(f"\nActor pairs appearing in at least {min_occurrences} movies together: {len(actor_pair_combined)}")
print("Top actor pairs by average score:")
print(actor_pair_combined.sort_values('average_score', ascending=False).head(10)[['actor1', 'actor2', 'count', 'average_score']])

# Analyze how actor combinations correlate with genres
actor_genre_counts = {}

for _, row in dataset_actors.iterrows():
    crew = row['crew']
    # Since genre is now one-hot encoded, we need to get genre names differently
    genres = [col for col in unique_genres if row.get(col, 0) == 1]
    
    for actor in crew:
        if actor not in actor_genre_counts:
            actor_genre_counts[actor] = Counter()
        for genre in genres:
            actor_genre_counts[actor][genre] += 1

# Find actors who specialize in specific genres
top_actors_by_genre = {}
for genre in unique_genres:
    top_actors = sorted([(actor, counts[genre]) for actor, counts in actor_genre_counts.items() if genre in counts], 
                        key=lambda x: x[1], reverse=True)[:5]
    if top_actors:  # Check if there are any actors for this genre
        top_actors_by_genre[genre] = top_actors

print("\nTop actors by genre:")
for genre, actors in top_actors_by_genre.items():
    if actors:  # Check if there are any actors for this genre
        print(f"{genre}: {', '.join([actor for actor, _ in actors])}")

Found 19 unique genres

Top 20 actor combinations by average score:
Al Pacino         Robert Duvall       86.500000
Robert Duvall     Diane Keaton        86.500000
Marlon Brando     Robert Duvall       85.000000
Elijah Wood       Andy Serkis         84.500000
                  Bernard Hill        84.500000
Ian McKellen      Bernard Hill        84.500000
Liv Tyler         Andy Serkis         84.500000
                  Bernard Hill        84.500000
Viggo Mortensen   Andy Serkis         84.500000
                  Bernard Hill        84.500000
Sean Astin        Andy Serkis         84.500000
                  Bernard Hill        84.500000
Andy Serkis       John Rhys-Davies    84.500000
                  Bernard Hill        84.500000
Cate Blanchett    Bernard Hill        84.500000
John Rhys-Davies  Bernard Hill        84.500000
Robert Duvall     G. D. Spradlin      84.500000
Elijah Wood       Ian McKellen        84.333333
                  Liv Tyler           84.333333
                  Vi

#### Korrelaatio arvioiden ja tuoton välillä. Voiko elokuva olla huonosti arvioitu mutta silti tuottoisa tai toisin päin?

                     score        profit
genre                                   
War              69.099644  1.307558e+08
Western          68.007634  1.372378e+08
Crime            65.129032  1.373403e+08
History          69.085511  1.381559e+08
Thriller         62.635177  1.460314e+08
Horror           59.807990  1.522667e+08
Drama            65.938190  1.653866e+08
Mystery          64.413953  1.671833e+08
Action           63.625091  1.802470e+08
Comedy           63.906091  1.832856e+08
Romance          62.573155  1.860445e+08
Science Fiction  63.396181  2.013719e+08
Adventure        65.266419  2.145588e+08
Music            68.996364  2.168855e+08
Fantasy          65.805091  2.241765e+08
Family           66.184604  2.367257e+08
Animation        69.044704  2.868047e+08
TV Movie         64.199052  3.736254e+08
Documentary      65.120603  4.223721e+08
category
Hyvin arvioitu, mutta vähän tuottoa    3400
Huonosti arvioitu, mutta tuottoisa     1521
Name: count, dtype: int64


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Kopioi aineisto
dataset_correlation = dataset.copy()

# Laske profit ja ROI
dataset_correlation['profit'] = dataset_correlation['revenue'] - dataset_correlation['budget_x']
dataset_correlation['roi'] = dataset_correlation['profit'] / dataset_correlation['budget_x']
# poista ylimääräiset välilyönnit
dataset_correlation['genre'] = dataset_correlation['genre'].str.strip().str.split(r'\s*,\s*')
# Erotellaan genret erillisiksi riveiksi
df = dataset_correlation.explode('genre')
genre_stats = df.groupby('genre')[['score', 'profit']].mean().sort_values(by='profit', ascending=True)
# Tulostetaan tulos
print(genre_stats)

# Laske keskiarvot
mean_score = dataset_correlation['score'].mean()
mean_revenue = dataset_correlation['revenue'].mean()

# Luo kategoriat
dataset_correlation['category'] = 'Muu'
dataset_correlation.loc[(dataset_correlation['score'] < mean_score) & (dataset_correlation['revenue'] > mean_revenue), 'category'] = 'Huonosti arvioitu, mutta tuottoisa'
dataset_correlation.loc[(dataset_correlation['score'] > mean_score) & (dataset_correlation['revenue'] < mean_revenue), 'category'] = 'Hyvin arvioitu, mutta vähän tuottoa'

# Poista 'Muu' kategoria
filtered_dataset = dataset_correlation[dataset_correlation['category'] != 'Muu']
# Tulosta kategorioiden jakauma
print(filtered_dataset['category'].value_counts())

# Visualisoi kategoriat
plt.figure(figsize=(10, 6))
sns.scatterplot(x='score', y='revenue', hue='category', data=filtered_dataset, palette='Set1', alpha=0.7)
plt.axhline(mean_revenue, color='red', linestyle='--', label='Keskiarvo (tuotto)')
plt.axvline(mean_score, color='blue', linestyle='--', label='Keskiarvo (arvosana)')
plt.title('Elokuvien luokittelu arvioiden ja tuoton mukaan')
plt.xlabel('IMDB-arvosana')
plt.ylabel('Tuotto (USD)')
plt.legend()
plt.show()

In [44]:
filtered_dataset = dataset_correlation[dataset_correlation['revenue'] > 10000]

#Huonosti arvioitu, mutta tuottoisin elokuva
max_revenue_low_score = filtered_dataset[filtered_dataset['category'] == 'Huonosti arvioitu, mutta tuottoisa'].nlargest(1, 'revenue')

print("Huonosti arvioitu, mutta tuottoisin elokuva:")
print(f"Arvosana: {max_revenue_low_score['score'].values[0]}")
print(f"Tuotto: {max_revenue_low_score['revenue'].values[0]} USD")
print(f"Nimi: {max_revenue_low_score['names'].values[0]}")
print("\n")

#Hyvin arvioitu, mutta vähiten tuottoinen elokuva
min_revenue_high_score = filtered_dataset[filtered_dataset['category'] == 'Hyvin arvioitu, mutta vähän tuottoa'].nsmallest(1, 'revenue')

print("Hyvin arvioitu, mutta vähiten tuottoinen elokuva:")
print(f"Arvosana: {min_revenue_high_score['score'].values[0]}")
print(f"Tuotto: {min_revenue_high_score['revenue'].values[0]} USD")
print(f"Nimi: {min_revenue_high_score['names'].values[0]}")

Huonosti arvioitu, mutta tuottoisin elokuva:
Arvosana: 44.0
Tuotto: 1515100211.0 USD
Nimi: The Avengers


Hyvin arvioitu, mutta vähiten tuottoinen elokuva:
Arvosana: 65.0
Tuotto: 11546.0 USD
Nimi: Sweat


#### Ennustamme elokuvan tuottoa annettujen parametrien perusteella

In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the dataset
df = dataset.copy()

# Calculate ROI
df['ROI'] = (df['revenue'] - df['budget_x']) / df['budget_x']

# Extract all individual genres from the genre column
all_genres = []
for genre_string in df['genre']:
    genres = [g.strip() for g in genre_string.split(',')]
    all_genres.extend(genres)

# Get unique genres and sort them
unique_genres = sorted(list(set(all_genres)))
print(f"Found {len(unique_genres)} unique genres")

# Create a new dataframe with a column for each genre
genres_df = pd.DataFrame(index=df.index)
for genre in unique_genres:
    genres_df[genre] = df['genre'].apply(lambda x: 1 if genre in x else 0)

# Concatenate the genres dataframe with the original dataframe
df = pd.concat([df, genres_df], axis=1)

# Extract actors from the crew column
df['actors'] = df['crew'].apply(lambda x: ', '.join([actor.split(',')[0] for actor in x]))
# [Tom cruise, Brad Pitt, Angelina Jolie] -> Tom Cruise, Brad Pitt, Angelina Jolie

# Limit the number of unique actors to the top N most frequent actors
top_n_actors = 100
actor_counts = df['actors'].value_counts()
top_actors = actor_counts.nlargest(top_n_actors).index
df['actors'] = df['actors'].apply(lambda x: x if x in top_actors else 'Other')

# Select features and target
features = unique_genres + ['actors', 'budget_x']
target = 'ROI'

# Define the preprocessing for the categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('actors', OneHotEncoder(handle_unknown='ignore'), ['actors']),
        ('budget', StandardScaler(), ['budget_x'])
    ],
    remainder='passthrough'
)

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with preprocessing and the model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Example usage
example_data1 = pd.DataFrame({
    'actors': ['Daniel Radcliffe'],
    'budget_x': [100000000]
})

example_data2 = pd.DataFrame({
    'actors': ['Tom Cruise'],
    'budget_x': [200000000]
})

for genre in unique_genres:
    example_data1[genre] = 0
example_data1['Drama'] = 1  # Set the appropriate genre(s) to 1

for genre in unique_genres:
    example_data2[genre] = 0
example_data2['Action'] = 1

example_data1['actors'] = example_data1['actors'].apply(lambda x: ', '.join([actor.split(',')[0] for actor in x.split(', ')]))
example_data2['actors'] = example_data2['actors'].apply(lambda x: ', '.join([actor.split(',')[0] for actor in x.split(', ')]))
example_pred = model.predict(example_data1[features])
print(f'Predicted ROI for a Drama movie with Daniel Radcliffe: {example_pred[0] * 100:.2f}%')

example_pred = model.predict(example_data2[features])
print(f'Predicted ROI for a Action movie with Tom Cruise: {example_pred[0] * 100:.2f}%')

Found 19 unique genres
Mean Squared Error: 1277462664.1060069
Predicted ROI for a Drama movie with Daniel Radcliffe: 179.89%
Predicted ROI for a Action movie with Tom Cruise: 126.77%


# Vaihe 5
### Analyysin tulkinta

#### 1. Mitkä genret ovat olleet suosittuja milloin?

##### Elokuvien määrän kasvu
Elokuvatuotanto on kasvanut räjähdysmäisesti – vuonna 2010 elokuvia tehtiin 1000-kertainen määrä vuoteen 1900 verrattuna.

##### Genrejen kehitys
Alkuvuosikymmeninä draama ja historialliset elokuvat hallitsivat. 1960-luvulta lähtien toiminta- ja seikkailuelokuvat yleistyivät, 1990-luvulla scifi ja fantasia kasvoivat CGI-teknologian myötä, ja 2010-luvulla suoratoistopalvelut lisäsivät dokumenttien ja indie-elokuvien suosiota.

##### Päätelmät
Elokuvateollisuuden kasvu ja teknologian kehitys ovat laajentaneet genrekirjoa. Toiminta ja seikkailu ovat nousseet suositummiksi, kun taas draama on säilynyt vakiintuneena.

#### 2. Millaisia ovat suositut näyttelijäparit?

##### Parhaat näyttelijäparit:
Al Pacino ja Robert Duvall saivat korkeimman keskiarvon (86.5). Myös Taru sormusten herrasta -elokuvien näyttelijät, kuten Elijah Wood ja Ian McKellen, erottuivat.

##### Useimmin yhdessä esiintyvät:
Frank Welker ja Grey DeLisle (33 elokuvaa, animaatio/dubbaukset) sekä Ikue Otani ja Megumi Hayashibara (anime).

##### Genreikoneita:
- Toiminta: Jackie Chan, Bruce Willis, Nicolas Cage
- Animaatio: Frank Welker, Grey DeLisle
- Draama: Robert De Niro, Tom Hanks, Keanu Reeves
- Scifi: Masako Nozawa, Megumi Hayashibara, Keanu Reeves, Scarlett Johansson

#### 3. Korrelaatio arvioiden ja tuoton välillä. Voiko elokuva olla huonosti arvioitu mutta silti tuottoisa tai toisin päin?

##### Keskeiset havainnot:
- 1521 elokuvaa olivat huonosti arvioituja mutta tuottoisia. Esim. The Avengers (arvosana: 44, tuotto: 1,5 miljardia USD)
- 3400 elokuvaa olivat hyvin arvioituja mutta vähätuottoisia. Esim. Sweat (Arvosana: 65, tuotto: 11546.0 USD)

##### Mitä visualisaatiosta voi lukea?

- Huonosti arvioidut, mutta tuottoisat elokuvat voivat olla blockbustereita, joiden vetovoima perustuu franchiseen, markkinointiin tai tähtinäyttelijöihin, vaikka arviot olisivat keskinkertaisia.
- Hyvin arvioidut, mutta vähätuottoiset elokuvat saattavat olla indie-elokuvia tai festivaalimenestyksiä, jotka eivät tavoita suurta yleisöä.

Erottelu auttaa ymmärtämään, että tuotto ei aina korreloi laadun kanssa, ja kaupallinen menestys voi johtua muista tekijöistä kuin kriitikoiden arvioista.

#### 4. Jos tekisimme elokuvan, millainen sen kannattaa olla, että se on tuottoisa ja suosittu?

Mallina käytettiin Random Forest Regressoria, joka ennustaa sijoitetun pääoman tuottoa (ROI) elokuvan budjetin, näyttelijöiden ja genren perusteella.

##### Keskeiset vaiheet:
- Muunnettiin genre ja näyttelijät numeeriseen muotoon
- Rajattiin näyttelijät 100 suosituimpaan
- Käytettiin One-Hot Encodingia kategorisille muuttujille
- Budjetti skaalattiin StandardScalerilla
- Data jaettiin 80/20-suhteessa testiin ja harjoitukseen

##### Mallin suorituskyky:
- Mean Squared Error: 1,28 miljardia

##### Esimerkkiennusteet:
- Draamaelokuva (Daniel Radcliffe): ROI 179.89%
- Toimintaelokuva (Tom Cruise): ROI 132.40%

# Vaihe 6
### Tulosten hyödyntäminen

#### Tutkimuksen tuloksia voidaan hyödyntää useilla tavoilla elokuvatuotannossa ja -markkinoinnissa:

##### 1. Tuotantopäätökset
- Blockbuster-strategia: Tuottajat voivat panostaa suuriin franchise-elokuviin, jotka tuottavat hyvin riippumatta kriitikoiden arvioista.
- Niche-markkinat: Pienemmät studiot voivat keskittyä laadukkaisiin, mutta vähätuottoisiin elokuviin, jotka saavat kriitikoiden ylistystä ja menestyvät esimerkiksi suoratoistopalveluissa.

##### 2. Markkinointistrategiat
- Heikosti arvioitujen, mutta tuottoisten elokuvien markkinointi voi keskittyä näyttäviin tehosteisiin, tunnetuihin näyttelijöihin ja laajaan mainoskampanjaan.
- Hyvin arvioiduille mutta vähätuottoisille elokuville voidaan kohdentaa markkinointia tarkemmin esimerkiksi elokuvafestivaaleihin, suoratoistopalveluihin tai arvosteluja arvostavalle yleisölle.

##### 3. Tulevien menestystekijöiden ennakointi

- Voidaan analysoida, mitkä genre- ja näyttelijäyhdistelmät ennustavat korkeaa tuottoa suhteessa budjettiin.
- Studion resurssit voidaan kohdistaa projekteihin, joilla on joko suuri kaupallinen potentiaali tai palkintomenestykseen tähtäävä profiili.

Ymmärtämällä arvostelujen ja tuottojen suhdetta, elokuvastudiot voivat tehdä strategisempia päätöksiä ja maksimoida joko kaupallisen tai taiteellisen menestyksen.